In [3]:
from langchain.tools import BaseTool
from transformers import BlipProcessor, BlipForConditionalGeneration, DetrImageProcessor, DetrForObjectDetection
from PIL import Image
import torch
#
import os
from tempfile import NamedTemporaryFile
from langchain.agents import initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

In [4]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
os.environ["AZURE_OPENAI_ENDPOINT"]=os.getenv("AZURE_OPENAI_ENDPOINT")

In [5]:
print(os.environ['OPENAI_API_KEY'])

188c17c4febd4ec08a28772d28edb00e


In [6]:
class ImageCaptionTool(BaseTool):
    name = "Image captioner"
    description = "Use this tool when given the path to an image that you would like to be described. " \
                  "It will return a simple caption describing the image."

    def _run(self, img_path):
        image = Image.open(img_path).convert('RGB')

        model_name = "Salesforce/blip-image-captioning-large"
        device = "cpu"  # cuda

        processor = BlipProcessor.from_pretrained(model_name)
        model = BlipForConditionalGeneration.from_pretrained(model_name).to(device)

        inputs = processor(image, return_tensors='pt').to(device)
        output = model.generate(**inputs, max_new_tokens=20)

        caption = processor.decode(output[0], skip_special_tokens=True)

        return caption

    def _arun(self, query: str):
        raise NotImplementedError("This tool does not support async")

In [7]:
class ObjectDetectionTool(BaseTool):
    name = "Object detector"
    description = "Use this tool when given the path to an image that you would like to detect objects. " \
                  "It will return a list of all detected objects. Each element in the list in the format: " \
                  "[x1, y1, x2, y2] class_name confidence_score."

    def _run(self, img_path):
        image = Image.open(img_path).convert('RGB')

        processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")
        model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50")

        inputs = processor(images=image, return_tensors="pt")
        outputs = model(**inputs)

        # convert outputs (bounding boxes and class logits) to COCO API
        # let's only keep detections with score > 0.9
        target_sizes = torch.tensor([image.size[::-1]])
        results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.9)[0]

        detections = ""
        for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
            detections += '[{}, {}, {}, {}]'.format(int(box[0]), int(box[1]), int(box[2]), int(box[3]))
            detections += ' {}'.format(model.config.id2label[int(label)])
            detections += ' {}\n'.format(float(score))

        return detections

    def _arun(self, query: str):
        raise NotImplementedError("This tool does not support async")

In [8]:
def get_image_caption(image_path):
    """
    Generates a short caption for the provided image.

    Args:
        image_path (str): The path to the image file.

    Returns:
        str: A string representing the caption for the image.
    """
    image = Image.open(image_path).convert('RGB')

    model_name = "Salesforce/blip-image-captioning-large"
    device = "cpu"  # cuda

    processor = BlipProcessor.from_pretrained(model_name)
    model = BlipForConditionalGeneration.from_pretrained(model_name).to(device)

    inputs = processor(image, return_tensors='pt').to(device)
    output = model.generate(**inputs, max_new_tokens=20)

    caption = processor.decode(output[0], skip_special_tokens=True)

    return caption


In [9]:
def detect_objects(image_path):
    """
    Detects objects in the provided image.

    Args:
        image_path (str): The path to the image file.

    Returns:
        str: A string with all the detected objects. Each object as '[x1, x2, y1, y2, class_name, confindence_score]'.
    """
    image = Image.open(image_path).convert('RGB')

    processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")
    model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50")

    inputs = processor(images=image, return_tensors="pt")
    outputs = model(**inputs)

    # convert outputs (bounding boxes and class logits) to COCO API
    # let's only keep detections with score > 0.9
    target_sizes = torch.tensor([image.size[::-1]])
    results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.9)[0]

    detections = ""
    for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
        detections += '[{}, {}, {}, {}]'.format(int(box[0]), int(box[1]), int(box[2]), int(box[3]))
        detections += ' {}'.format(model.config.id2label[int(label)])
        detections += ' {}\n'.format(float(score))

    return detections

In [14]:
import openai
from getpass import getpass
#set the openai_api_key
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
os.environ["AZURE_OPENAI_ENDPOINT"]=os.getenv("AZURE_OPENAI_ENDPOINT")

In [13]:
from langchain_core.messages import HumanMessage
from langchain_openai import AzureChatOpenAI

In [15]:
#initialize the gent
tools = [ImageCaptionTool(), ObjectDetectionTool()]

conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)
"""
llm = ChatOpenAI(
    openai_api_key= os.environ['OPENAI_API_KEY'],
    temperature=0,
    model_name="gpt-35-turbo"
)
"""

llm = AzureChatOpenAI(
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
)

agent = initialize_agent(
    agent="chat-conversational-react-description",
    tools=tools,
    llm=llm,
    max_iterations=5,
    verbose=True,
    memory=conversational_memory,
    early_stopping_method='generate'
)

C:\Users\Student\AppData\Roaming\Python\Python311\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [16]:
image_path = "C:\\Users\\Langchain\\Updated-Langchain\\images\\Parsons_PR.jpg"
user_question = "generate a caption for this iamge?"
response = agent.run(f'{user_question}, this is the image path: {image_path}')
print(response)

C:\Users\Student\AppData\Roaming\Python\Python311\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
{
    "action": "Image captioner",
    "action_input": "C:\\Users\\Langchain\\Updated-Langchain\\images\\Parsons_PR.jpg"
}
Observation: cars are driving down the street in traffic at a green light
Thought:{
    "action": "Final Answer",
    "action_input": "The tool responded with the following caption for the given image: 'cars are driving down the street in traffic at a green light'."
}

> Finished chain.
The tool responded with the following caption for the given image: 'cars are driving down the street in traffic at a green light'.


In [17]:
image_path = "C:\\Users\\Langchain\\Updated-Langchain\\images\\Parsons_PR.jpg"
user_question = "Please tell me what are the items present in the image."
response = agent.run(f'{user_question}, this is the image path: {image_path}')
print(response)



> Entering new AgentExecutor chain...
{
    "action": "Object detector",
    "action_input": "C:\\Users\\Langchain\\Updated-Langchain\\images\\Parsons_PR.jpg"
}

C:\Users\Student\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Student\.cache\huggingface\hub\models--timm--resnet50.a1_in1k. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing


Observation: [518, 40, 582, 110] car 0.9300956726074219
[188, 381, 311, 469] car 0.9253566861152649
[1068, 223, 1104, 342] person 0.9871609210968018
[828, 233, 949, 329] car 0.9450383186340332
[1076, 263, 1106, 347] bicycle 0.9070357084274292
[635, 71, 713, 135] car 0.9211896657943726
[0, 433, 100, 603] car 0.9781932830810547
[151, 747, 339, 799] car 0.9839041233062744
[389, 267, 493, 367] car 0.980132520198822
[192, 478, 341, 633] car 0.9953184723854065
[578, 117, 828, 550] traffic light 0.9860708117485046
[802, 666, 1028, 798] car 0.982887327671051
[0, 639, 84, 799] car 0.9630057215690613
[1057, 608, 1199, 766] car 0.9652812480926514
[988, 218, 1031, 347] person 0.947163462638855
[751, 524, 909, 675] car 0.9911808371543884
[489, 560, 670, 749] car 0.9970001578330994

Thought:{
    "action": "Final Answer",
    "action_input": "The following objects are present in the image: car, person, bicycle, traffic light."
}

> Finished chain.
The following objects are present in the image: car

In [18]:
image_path = "C:\\Users\\Langchain\\Updated-Langchain\\images\\Parsons_PR.jpg"
user_question = "Please tell me the bounding boxes of all detected objects in the image."
response = agent.run(f'{user_question}, this is the image path: {image_path}')
print(response)



> Entering new AgentExecutor chain...
```json
{
    "action": "Object detector",
    "action_input": "C:\\Users\\Langchain\\Updated-Langchain\\images\\Parsons_PR.jpg"
}
```

Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



Observation: [518, 40, 582, 110] car 0.9300956726074219
[188, 381, 311, 469] car 0.9253566861152649
[1068, 223, 1104, 342] person 0.9871609210968018
[828, 233, 949, 329] car 0.9450383186340332
[1076, 263, 1106, 347] bicycle 0.9070357084274292
[635, 71, 713, 135] car 0.9211896657943726
[0, 433, 100, 603] car 0.9781932830810547
[151, 747, 339, 799] car 0.9839041233062744
[389, 267, 493, 367] car 0.980132520198822
[192, 478, 341, 633] car 0.9953184723854065
[578, 117, 828, 550] traffic light 0.9860708117485046
[802, 666, 1028, 798] car 0.982887327671051
[0, 639, 84, 799] car 0.9630057215690613
[1057, 608, 1199, 766] car 0.9652812480926514
[988, 218, 1031, 347] person 0.947163462638855
[751, 524, 909, 675] car 0.9911808371543884
[489, 560, 670, 749] car 0.9970001578330994

Thought:```json
{
    "action": "Final Answer",
    "action_input": "The detected objects in the image are: car, person, bicycle, traffic light. Below are the bounding boxes coordinates and confidence scores for each ob